In [1]:
#tbats, history since August 2015

#importing necessary forecasting tools from sktime
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.tbats import TBATS

#import pandas library for data analysis & manipulation
import pandas as pd
#import pandas library for scientific computing
import numpy as np


#training data for model
train_path = "***\\Project_Data\\train.csv.zip"
#test data for prediction model
test_path = "***\\Project_Data\\test.csv"
#output path for submission
forecast_path = "***\\Project_Models\\tbats\\submission.csv"

#import data into pandas dataframe
train_data = pd.read_csv(train_path, 
                         usecols=["store_nbr", "family", "date", "sales"],
                         dtype={"store_nbr":"int",
                                "family":"object",
                                "sales":"float32",},
                                parse_dates=["date"],
                                infer_datetime_format=True,
                                compression="zip")
test_data = pd.read_csv(test_path, 
                         usecols=["id","store_nbr", "family", "date"],
                         dtype={"id":"int",
                                "store_nbr":"int",
                                "family":"object",
                                "sales":"float32",},
                                parse_dates=["date"],
                                infer_datetime_format=True)
                            
#check number of rows & columns in each set
print("training set",train_data.shape)
print("testing set", test_data.shape)

training set (3000888, 4)
testing set (28512, 4)


In [2]:
#sktime algorithm requires explicit declaration of frequency
train_data["date"] = train_data.date.dt.to_period("D")
test_data["date"] = test_data.date.dt.to_period("D")

In [3]:
#review training data
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 4 columns):
 #   Column     Dtype    
---  ------     -----    
 0   date       period[D]
 1   store_nbr  int32    
 2   family     object   
 3   sales      float32  
dtypes: float32(1), int32(1), object(1), period[D](1)
memory usage: 68.7+ MB


In [4]:
#peek at detail of training data
train_data

,date,store_nbr,family,sales
0,2013-01-01,1,AUTOMOTIVE,0.000000
1,2013-01-01,1,BABY CARE,0.000000
2,2013-01-01,1,BEAUTY,0.000000
3,2013-01-01,1,BEVERAGES,0.000000
4,2013-01-01,1,BOOKS,0.000000
...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.132996
3000884,2017-08-15,9,PREPARED FOODS,154.552994
3000885,2017-08-15,9,PRODUCE,2419.729004
3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000000


In [5]:
#review test data
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype    
---  ------     --------------  -----    
 0   id         28512 non-null  int32    
 1   date       28512 non-null  period[D]
 2   store_nbr  28512 non-null  int32    
 3   family     28512 non-null  object   
dtypes: int32(2), object(1), period[D](1)
memory usage: 668.4+ KB


In [6]:
test_data.head()

,id,date,store_nbr,family
0,3000888,2017-08-16,1,AUTOMOTIVE
1,3000889,2017-08-16,1,BABY CARE
2,3000890,2017-08-16,1,BEAUTY
3,3000891,2017-08-16,1,BEVERAGES
4,3000892,2017-08-16,1,BOOKS


In [7]:
#prepare data
train_data = train_data.sort_values(by=["store_nbr", "family", "date"])

In [8]:
#selecting subset of data
train_sub = train_data.loc[(train_data.date > "2015-08-01")]
train_sub

,date,store_nbr,family,sales
1676862,2015-08-02,1,AUTOMOTIVE,0.0
1678644,2015-08-03,1,AUTOMOTIVE,5.0
1680426,2015-08-04,1,AUTOMOTIVE,4.0
1682208,2015-08-05,1,AUTOMOTIVE,17.0
1683990,2015-08-06,1,AUTOMOTIVE,4.0
...,...,...,...,...
2993627,2017-08-11,54,SEAFOOD,0.0
2995409,2017-08-12,54,SEAFOOD,1.0
2997191,2017-08-13,54,SEAFOOD,2.0
2998973,2017-08-14,54,SEAFOOD,0.0


In [9]:
#sktime forecasting require hierarchical multi-index in pandas dataframe
train_sub.set_index(["store_nbr", "family", "date"], inplace=True)

In [10]:
#checking results
train_sub

sales
store_nbr family     date             
1         AUTOMOTIVE 2015-08-02    0.0
                     2015-08-03    5.0
                     2015-08-04    4.0
                     2015-08-05   17.0
                     2015-08-06    4.0
...                                ...
54        SEAFOOD    2017-08-11    0.0
                     2017-08-12    1.0
                     2017-08-13    2.0
                     2017-08-14    0.0
                     2017-08-15    3.0

[1324026 rows x 1 columns]

In [11]:
#checking types
train_sub.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1324026 entries, (1, 'AUTOMOTIVE', Period('2015-08-02', 'D')) to (54, 'SEAFOOD', Period('2017-08-15', 'D'))
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   sales   1324026 non-null  float32
dtypes: float32(1)
memory usage: 10.1+ MB


In [12]:
#verifying timepoint index landed where expected
train_sub.index.get_level_values(level=-1)

PeriodIndex(['2015-08-02', '2015-08-03', '2015-08-04', '2015-08-05',
             '2015-08-06', '2015-08-07', '2015-08-08', '2015-08-09',
             '2015-08-10', '2015-08-11',
             ...
             '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
             '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
             '2017-08-14', '2017-08-15'],
            dtype='period[D]', name='date', length=1324026)

In [13]:
#setting desired forecasting range
horizon = ForecastingHorizon(pd.PeriodIndex(pd.date_range("2017-08-16", periods=16, freq="D")), is_relative=False)
horizon

ForecastingHorizon(['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
             '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
             '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
             '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31'],
            dtype='period[D]', is_relative=False)

In [14]:
#set forecaster
forecaster = TBATS(use_box_cox=False, use_trend=False, sp=7, use_arma_errors=False)

In [15]:
#run prediction
next15 = forecaster.fit(train_sub, fh=horizon).predict()

In [16]:
#check results
next15

sales
store_nbr family     date                
1         AUTOMOTIVE 2017-08-16  4.653074
                     2017-08-17  5.262448
                     2017-08-18  5.004559
                     2017-08-19  4.658452
                     2017-08-20  4.357469
...                                   ...
54        SEAFOOD    2017-08-27  2.830348
                     2017-08-28  3.180842
                     2017-08-29  3.247268
                     2017-08-30  2.861994
                     2017-08-31  2.287405

[28512 rows x 1 columns]

In [17]:
#resetting as normal pandas frame
next15_now = next15.reset_index()

In [18]:
#looking at my results
next15_now

,store_nbr,family,date,sales
0,1,AUTOMOTIVE,2017-08-16,4.653074
1,1,AUTOMOTIVE,2017-08-17,5.262448
2,1,AUTOMOTIVE,2017-08-18,5.004559
3,1,AUTOMOTIVE,2017-08-19,4.658452
4,1,AUTOMOTIVE,2017-08-20,4.357469
...,...,...,...,...
28507,54,SEAFOOD,2017-08-27,2.830348
28508,54,SEAFOOD,2017-08-28,3.180842
28509,54,SEAFOOD,2017-08-29,3.247268
28510,54,SEAFOOD,2017-08-30,2.861994


In [19]:
#adding predictions to test data
test_fc = pd.merge(test_data, next15_now, left_on=["store_nbr", "family", "date"], right_on=["store_nbr", "family", "date"])

In [20]:
#checking results
test_fc

,id,date,store_nbr,family,sales
0,3000888,2017-08-16,1,AUTOMOTIVE,4.653074
1,3000889,2017-08-16,1,BABY CARE,0.000000
2,3000890,2017-08-16,1,BEAUTY,3.787182
3,3000891,2017-08-16,1,BEVERAGES,2340.625086
4,3000892,2017-08-16,1,BOOKS,0.063560
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,333.925719
28508,3029396,2017-08-31,9,PREPARED FOODS,100.526515
28509,3029397,2017-08-31,9,PRODUCE,1417.524281
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,145.505171


In [21]:
#formatting for submission
forecast = pd.DataFrame({"id":test_fc.id, "sales": test_fc.sales})

In [22]:
#checking results
forecast

,id,sales
0,3000888,4.653074
1,3000889,0.000000
2,3000890,3.787182
3,3000891,2340.625086
4,3000892,0.063560
...,...,...
28507,3029395,333.925719
28508,3029396,100.526515
28509,3029397,1417.524281
28510,3029398,145.505171


In [23]:
#exporting to csv, dropping index
forecast.to_csv(forecast_path, index=False)